# 📊 Clutch Revenue: Predicción del Valor Comercial en la NBA

**Objetivo:**
- Predecir el valor comercial potencial de un jugador NBA en función de su rendimiento “clutch” (momentos críticos del juego) y su visibilidad mediática, para optimizar decisiones de marketing, patrocinios y fichajes.
- Contestar, desde la lógica de negocio, las preguntas clave:
  1. ¿Qué significa “jugador clutch” y cómo lo medimos de forma cuantitativa?
  2. ¿Qué entendemos por “valor comercial”? ¿Ingreso directo (entradas, camisetas), impacto en patrocinio o valor proyectado del contrato?
  3. ¿Puede este modelo extenderse a otras ligas o categorías?

---

## 1. ¿Qué archivos leer y por qué?

Para nuestro flujo de trabajo **de limpieza y análisis**, nos centraremos en **estos CSV**:

1. **`play_by_play.csv`**
   - Contiene cada jugada de todos los partidos.
   - Crucial para **identificar jugadas “clutch”** (últimos 5 minutos y marcador cerrado).

2. **`game.csv`** (o `game_info.csv`)
   - Datos de cada partido: fecha, equipos, local/visitante, temporada, etc.
   - Necesario para saber en qué año jugó cada jugada, agrupar por temporada, y contextualizar.

3. **`player.csv`** (o `common_player_info.csv`)
   - Información estática de cada jugador: nombre, equipo, posición, fecha de nacimiento, etc.
   - Se usa para **asignar nombre/edad/equipo** a cada `player_id`.

4. **`other_stats.csv`** (o `box_scores.csv`)
   - Estadísticas acumuladas de cada jugador en cada partido (puntos, rebotes, asistencias, minutos, etc.).
   - Nos sirve para calcular métricas globales de rendimiento (por ejemplo, Win Shares, minutos totales en clutch, etc.) y para estudiar correlaciones con el clutch performance.

> **Nota:** Archivos como `team.csv`, `line_score.csv`, `draft_history.csv`, `inactive_players.csv`, `officials.csv` no son prioritarios para la métrica clutch+valor comercial. Podemos ignorarlos en esta fase.

---

## 2. Estructura del Notebook

A continuación encontrarás un **notebook** dividido en celdas Markdown y código (`Python + Pandas`), paso a paso:

1. **Introducción y Objetivos** (Markdown)
2. **Carga Parcial de Archivos (solo columnas necesarias)**
3. **Filtrado de Jugadas Clutch**
4. **Cálculo de Estadísticas Clutch por Jugador**
5. **Definición de “Valor Comercial” y creación de proxies**
6. **Construcción de la Tabla Final (métricas + proxies)**
7. **Visualizaciones y Storytelling**
8. **Respuestas de Negocio en el Notebook**
9. **Extender a Otras Ligas / Categorías** (Markdown)

---



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [2]:
preview_pb = pd.read_csv("play_by_play.csv", nrows=5)
preview_g  = pd.read_csv("game.csv", nrows=5)
preview_p  = pd.read_csv("player.csv", nrows=5)
preview_os = pd.read_csv("other_stats.csv", nrows=5)

print("Columnas play_by_play.csv:", preview_pb.columns.tolist())
print("Columnas game.csv:", preview_g.columns.tolist())
print("Columnas player.csv:", preview_p.columns.tolist())
print("Columnas other_stats.csv:", preview_os.columns.tolist())


Columnas play_by_play.csv: ['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period', 'wctimestring', 'pctimestring', 'homedescription', 'neutraldescription', 'visitordescription', 'score', 'scoremargin', 'person1type', 'player1_id', 'player1_name', 'player1_team_id', 'player1_team_city', 'player1_team_nickname', 'player1_team_abbreviation', 'person2type', 'player2_id', 'player2_name', 'player2_team_id', 'player2_team_city', 'player2_team_nickname', 'player2_team_abbreviation', 'person3type', 'player3_id', 'player3_name', 'player3_team_id', 'player3_team_city', 'player3_team_nickname', 'player3_team_abbreviation', 'video_available_flag']
Columnas game.csv: ['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home', 'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home', 'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_hom

# 📊 Clutch Revenue: Predicción del Valor Comercial en la NBA

**Objetivo principal**
- Crear un notebook que limpie y prepare los datos necesarios para predecir el valor comercial potencial de un jugador NBA basándonos en su rendimiento “clutch” (momentos críticos del juego) y su visibilidad.
- A lo largo del notebook responderemos, desde la lógica de negocio, las siguientes preguntas:
  1. **¿Qué significa “jugador clutch” y cómo lo cuantificamos?**
  2. **¿Qué entendemos por “valor comercial”?** ¿Ingreso directo (entradas, camisetas), impacto en patrocinio o valor proyectado del contrato?
  3. **¿Se puede extender este modelo a otras ligas o categorías?**

---

## 1. Archivos a utilizar

Para este proyecto necesitamos únicamente los archivos que contienen información a nivel de jugada, partido y jugador. Los demás archivos (equipos, oficiales, draft, etc.) no son necesarios en esta fase de limpieza y definición de métricas.

Los CSV que **leeremos** son:

1. **`play_by_play.csv`**
   - Cada fila es un evento/jugada de un partido.
   - Columnas clave:
     - `game_id`
     - `eventmsgtype` (1 = tiro convertido, 2 = tiro fallado, 3 = rebote, …)
     - `period` (1,2,3,4,5(OT), …)
     - `wctimestring` (tiempo de juego restante en formato “MM:SS”)
     - `scoremargin` (diferencia de puntos, positivo si el equipo local va arriba)
     - `player1_id`, `player1_name` (identifica al jugador que protagoniza el evento)
     - `homedescription`, `visitordescription` (texto descriptivo de la jugada, útil para saber cuántos puntos)

2. **`game.csv`** (o `game_info.csv`)
   - Metadatos de cada partido.
   - Columnas clave:
     - `game_id` (para hacer merge con play_by_play)
     - `season_id` (temporada, ej. “22019” para 2019-20)
     - `game_date` (fecha real del partido)
     - `team_id_home`, `team_id_away` (IDs de equipos)
     - `pts_home`, `pts_away` (puntos finales)
     - Otros campos de box score a nivel de equipo (opcionales).

3. **`player.csv`** (o `common_player_info.csv`)
   - Información estática de cada jugador.
   - Columnas clave:
     - `id` (igual a `player1_id` en play_by_play)
     - `full_name`, `first_name`, `last_name`
     - `is_active` (si sigue activo al momento de la extracción)

4. **`other_stats.csv`** (o `box_scores.csv`)
   - Estadísticas agregadas por partido y equipo (no por jugador individual).
   - No es imprescindible para métricas clutch, pero lo cargamos para enriquecer el contexto (puntos en pintura, rebotes de equipo, turnovers totales).

---

## 2. Carga y limpieza inicial de los datos 🔍

### 2.1. Importar librerías

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opcional: si quieres mostrar gráficos inline
%matplotlib inline


In [3]:
# 2.2.1. Definir columnas y tipos para play_by_play
cols_plays = [
    "game_id",
    "eventmsgtype",
    "period",
    "wctimestring",
    "scoremargin",
    "player1_id",
    "player1_name",
    "homedescription",
    "visitordescription"
]

dtypes_plays = {
    "game_id": "int32",
    "eventmsgtype": "uint8",
    "period": "uint8",
    "scoremargin": "int8",       # Diferencia de puntos: suele estar entre -40 y +40
    "player1_id": "int32",
    "player1_name": "object",
    "homedescription": "object",
    "visitordescription": "object"
    # wctimestring lo mantendremos string (object)
}

# 2.2.2. Definir columnas y tipos para game.csv
cols_games = [
    "game_id",
    "season_id",
    "game_date",
    "team_id_home",
    "team_id_away",
    "pts_home",
    "pts_away"
]

dtypes_games = {
    "game_id": "int32",
    "season_id": "object",      # Ej. “22019”
    "team_id_home": "int16",
    "team_id_away": "int16",
    "pts_home": "int16",
    "pts_away": "int16"
}

# 2.2.3. Definir columnas y tipos para player.csv (pequeño, carga completa)
cols_players = ["id", "full_name", "first_name", "last_name", "is_active"]
dtypes_players = {
    "id": "int32",
    "full_name": "object",
    "first_name": "object",
    "last_name": "object",
    "is_active": "bool"
}

# 2.2.4. Definir columnas y tipos para other_stats.csv
#   Sólo algunas columnas a modo de ejemplo (puntos en pintura, turnovers, rebotes)
cols_other = [
    "game_id",
    "team_id_home",
    "team_id_away",
    "pts_paint_home",
    "pts_paint_away",
    "team_turnovers_home",
    "team_turnovers_away",
    "team_rebounds_home",
    "team_rebounds_away"
]

dtypes_other = {
    "game_id": "int32",
    "team_id_home": "int16",
    "team_id_away": "int16",
    "pts_paint_home": "int16",
    "pts_paint_away": "int16",
    "team_turnovers_home": "int16",
    "team_turnovers_away": "int16",
    "team_rebounds_home": "int16",
    "team_rebounds_away": "int16"
}
